In [1]:
import os
import pandas as pd
import re
import itertools

In [12]:
folder_path = 'Techstacks'

files = []
for file in os.listdir(folder_path):
    
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep=';', header=None, names=['Link', 'Branch', 'Techstack'])
    files.append(df)
    
df = pd.concat(files, ignore_index=True)
df.drop_duplicates(inplace=True)
df.shape

(9950, 3)

### Technologies

In [23]:
technologies = df['Techstack'].to_list()
technologies = [tech.split(',') for tech in technologies]
technologies = list(itertools.chain.from_iterable(technologies))
technologies = [re.sub(r'[^A-Za-z0-9. ]', '', tech).strip() for tech in technologies]
technologies = list(set(technologies))
len(technologies)

315

### Categories

In [24]:
from Wappalyzer import Wappalyzer
wappalyzer = Wappalyzer.latest()

/home/marco/anaconda3/lib/python3.11/site-packages/Wappalyzer/Wappalyzer.py:226: UserWarning: Caught 'unbalanced parenthesis at position 119' compiling regex: ['(?:<div class="sf-toolbar[^>]+?>[^]+<span class="sf-toolbar-value">([\\d.])+|<div id="sfwdt[^"]+" class="[^"]*sf-toolbar)', 'version:\\1']
  warnings.warn(


In [25]:
categories = []

for technology in technologies:
    
    category = wappalyzer.get_categories(technology)
    
    for i in category:
        if i in categories:
            continue
        
        categories.append(i)
        
len(categories)

40

### Database

In [26]:
import sqlite3
con = sqlite3.connect('data.db')
cur = con.cursor()

In [27]:
# Schema creation
cur.execute(
    """CREATE TABLE Category (
    ID INTEGER PRIMARY KEY,
    Name VARCHAR(128) NOT NULL
    )
    """
)

cur.execute(
    """CREATE TABLE Technology (
    ID INTEGER PRIMARY KEY,
    Name VARCHAR(128) NOT NULL
    )
    """
)

cur.execute(
    """CREATE TABLE Website (
    ID INTEGER PRIMARY KEY,
    Link VARCHAR(255) NOT NULL,
    Branche VARCHAR(128) NOT NULL
    )
    """
)

cur.execute(
    """CREATE TABLE Technology_Category (
    TechnologyID INTEGER NOT NULL,
    CategoryID INTEGER NOT NULL,
    PRIMARY KEY(TechnologyID, CategoryID),
    FOREIGN KEY(TechnologyID) REFERENCES Technology(ID) ON DELETE CASCADE,
    FOREIGN KEY(CategoryID) REFERENCES Category(ID) ON DELETE CASCADE
    )
    """
)

cur.execute(
    """CREATE TABLE Website_Technology (
    WebsiteID INTEGER NOT NULL,
    TechnologyID INTEGER NOT NULL,
    PRIMARY KEY(WebsiteID, TechnologyID),
    FOREIGN KEY(WebsiteID) REFERENCES Website(ID) ON DELETE CASCADE,
    FOREIGN KEY(TechnologyID) REFERENCES Technology(ID) ON DELETE CASCADE
    )
    """
)

#### Website Table

In [29]:
links = df['Link'].to_list()
branch = df['Branch'].to_list()

for i in range(len(links)):
    cur.execute("""INSERT INTO Website VALUES(NULL, ?, ?)""", (links[i], branch[i]))
    
con.commit()

#### Technology Table

In [30]:
for technology in technologies:
    cur.execute("""INSERT INTO Technology VALUES(NULL, ?)""", (technology,))
    
con.commit()

#### Category Table

In [31]:
for category in categories:
    cur.execute("""INSERT INTO Category VALUES(NULL, ?)""", (category,))
    
con.commit()

#### Technology_Category Table

In [32]:
for technology in technologies:
    
    technology_id = technologies.index(technology)+1
    technology_categories = wappalyzer.get_categories(technology)
    
    if technology_categories:
        for category in technology_categories:
            category_id = categories.index(category)+1
            cur.execute("""INSERT INTO Technology_Category VALUES(?, ?)""", (technology_id, category_id))       
    
con.commit()

#### Website_Technology Table

In [33]:
techstacks = df['Techstack'].to_list()
techstacks = [stack.split(',') for stack in techstacks]
techstacks = [[re.sub(r'[^A-Za-z0-9. ]', '', tech).strip() for tech in stack] for stack in techstacks]

In [34]:
for i in range(len(links)):
    
    website_id = i+1
    website_stack = techstacks[i]
    
    for technology in website_stack:
        
        technology_id = technologies.index(technology)+1
        
        cur.execute("""INSERT INTO Website_Technology VALUES(?, ?)""", (website_id, technology_id))
    
con.commit()   